## 1. Setup and Libraries

To build a chatbot with natural language processing (NLP) capabilities, we will use the following Python libraries:

- **spaCy:** For tokenization, parsing, and entity recognition.
  
- **NLTK (Natural Language Toolkit):** For additional NLP tasks like stopword removal or stemming (optional).

- **Transformers (Hugging Face):** For more advanced language models like BERT or GPT.

- **pandas:** For handling and querying the financial data.

- **Flask (optional):** If you want to deploy your chatbot later.

## 2. Loading and Structuring Financial Data
First, you need to load your financial data (growth rates, profitability ratios, etc.) into a format that the chatbot can access.

**Step: Load Data into DataFrames**

In [1]:
import pandas as pd

# Load your data into pandas DataFrames
growth_data = pd.DataFrame({
    'Date': ['2021-09-25', '2022-09-24', '2023-09-30', '2021-06-30', '2022-06-30', '2023-06-30', '2021-12-31', '2022-12-31', '2023-12-31'],
    'Company': ['Apple Inc.', 'Apple Inc.', 'Apple Inc.', 'Microsoft', 'Microsoft', 'Microsoft', 'Tesla', 'Tesla', 'Tesla'],
    'Assets Growth (%)': [None, 0.4994, -0.0488, None, 9.3059, 12.9196, None, 32.5232, 29.4882],
    'Cash Flow Growth (%)': [None, 17.41, -9.503, None, 16.0216, -1.6319, None, 28.0682, -9.9701],
    'Liabilities Growth (%)': [None, 4.922, -3.8552, None, 3.3928, 3.7595, None, 19.2877, 18.0269],
    'Net Income Growth (%)': [None, 5.4109, -2.8135, None, 18.7152, -0.5183, None, 127.505, 19.4409],
    'Revenue Growth (%)': [None, 7.7938, -2.8005, None, 17.9561, 6.882, None, 51.3517, 18.7953]
})

ratios_data = pd.DataFrame({
    'Date': ['2021-09-25', '2022-09-24', '2023-09-30', '2021-06-30', '2022-06-30', '2023-06-30', '2021-12-31', '2022-12-31', '2023-12-31'],
    'Company': ['Apple Inc.', 'Apple Inc.', 'Apple Inc.', 'Microsoft', 'Microsoft', 'Microsoft', 'Tesla', 'Tesla', 'Tesla'],
    'Net Profit Margin (%)': [25.88, 25.31, 25.30, 36.45, 36.68, 34.15, 10.25, 15.41, 15.50],
    'Return on Assets (ROA) (%)': [26.97, 28.29, 27.51, 18.36, 19.94, 17.56, 8.88, 15.25, 14.07],
    'Debt to Assets Ratio (%)': [82.03, 85.64, 82.37, 57.46, 54.35, 49.94, 49.17, 44.26, 40.34],
    'Equity to Assets Ratio (%)': [17.97, 14.36, 17.63, 42.54, 45.65, 50.06, 50.83, 55.74, 59.66],
    'Debt to Equity Ratio': [4.56, 5.96, 4.67, 1.35, 1.19, 0.99, 0.97, 0.79, 0.68],
    'Operating Cash Flow to Revenue Ratio (%)': [28.44, 30.98, 28.84, 45.65, 44.91, 41.33, 21.36, 18.07, 13.70],
})


In [2]:
display(growth_data)

,Date,Company,Assets Growth (%),Cash Flow Growth (%),Liabilities Growth (%),Net Income Growth (%),Revenue Growth (%)
0,2021-09-25,Apple Inc.,NaN,NaN,NaN,NaN,NaN
1,2022-09-24,Apple Inc.,0.4994,17.4100,4.9220,5.4109,7.7938
2,2023-09-30,Apple Inc.,-0.0488,-9.5030,-3.8552,-2.8135,-2.8005
3,2021-06-30,Microsoft,NaN,NaN,NaN,NaN,NaN
4,2022-06-30,Microsoft,9.3059,16.0216,3.3928,18.7152,17.9561
5,2023-06-30,Microsoft,12.9196,-1.6319,3.7595,-0.5183,6.8820
6,2021-12-31,Tesla,NaN,NaN,NaN,NaN,NaN
7,2022-12-31,Tesla,32.5232,28.0682,19.2877,127.5050,51.3517
8,2023-12-31,Tesla,29.4882,-9.9701,18.0269,19.4409,18.7953


In [3]:
display(ratios_data)

,Date,Company,Net Profit Margin (%),Return on Assets (ROA) (%),Debt to Assets Ratio (%),Equity to Assets Ratio (%),Debt to Equity Ratio,Operating Cash Flow to Revenue Ratio (%)
0,2021-09-25,Apple Inc.,25.88,26.97,82.03,17.97,4.56,28.44
1,2022-09-24,Apple Inc.,25.31,28.29,85.64,14.36,5.96,30.98
2,2023-09-30,Apple Inc.,25.30,27.51,82.37,17.63,4.67,28.84
3,2021-06-30,Microsoft,36.45,18.36,57.46,42.54,1.35,45.65
4,2022-06-30,Microsoft,36.68,19.94,54.35,45.65,1.19,44.91
5,2023-06-30,Microsoft,34.15,17.56,49.94,50.06,0.99,41.33
6,2021-12-31,Tesla,10.25,8.88,49.17,50.83,0.97,21.36
7,2022-12-31,Tesla,15.41,15.25,44.26,55.74,0.79,18.07
8,2023-12-31,Tesla,15.50,14.07,40.34,59.66,0.68,13.70


## 3. Building the NLP Pipeline

**Tokenization and Named Entity Recognition (NER)**
We will start by using spaCy for tokenization and NER. This will help the chatbot recognize financial terms and company names in queries.

**Step: Load spaCy and Define NLP Pipeline**

In [4]:
import spacy

# Load spaCy's English NLP model
nlp = spacy.load("en_core_web_sm")

# Normalize text (remove special characters like curly apostrophes)
def normalize_text(text):
    return text.replace("’", "'")

# List of known companies
known_companies = ["Tesla", "Apple", "Microsoft"]

# Extract financial terms (dynamically from your DataFrame columns)
growth_columns = [col for col in growth_data.columns if col not in ['Company', 'Date']]
ratios_columns = [col for col in ratios_data.columns if col not in ['Company', 'Date']]
all_columns = growth_columns + ratios_columns
financial_terms = [col.replace("_", " ").replace("(%)", "").strip().lower() for col in all_columns]

**Custom Entity and Financial Term Matching**
For financial chatbot use cases, you’ll likely need custom matching for terms like “revenue growth”, “net income”, etc. You can use spaCy’s PhraseMatcher or EntityRuler for this.
**Steps to Generate Financial Terms from DataFrames:**
1. Extract the column names from both DataFrames.
2. Clean the column names to generate readable financial terms by removing symbols like (%), and replacing underscores (_) with spaces.
3. Convert them to a list of financial terms that can be used in your PhraseMatcher.

In [5]:
from spacy.matcher import PhraseMatcher
# Initialize the matcher for financial terms
matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp.make_doc(term) for term in financial_terms]
matcher.add("FIN_TERMS", patterns)

# Function to extract entities from the query
def extract_entities_from_query(query):
    # Normalize the query to handle apostrophes
    query = normalize_text(query)
    
    # Process query through the NLP pipeline
    doc = nlp(query)
    
    # Initialize variables
    company = None
    year = None
    metric = None
    
    # Identify company using a list of known companies
    for token in doc:
        if token.text in known_companies:
            company = token.text
    
    # Identify year using spaCy's entity recognition (DATE label)
    for ent in doc.ents:
        if ent.label_ == 'DATE':  # Year
            year = ent.text.strip()
    
    # Use PhraseMatcher to find the financial term (metric) in the query
    matches = matcher(doc)
    for match_id, start, end in matches:
        metric = doc[start:end].text.lower()  # Match the financial term
    
    # Debugging: print what we extracted
    print(f"Extracted company: {company}, metric: {metric}, year: {year}")
    
    return company, metric, year

## 4. Query Parsing and Response Generation

Once we’ve tokenized and matched key financial terms, the chatbot needs to fetch the relevant data from your DataFrames.

**Step:** Mapping Queries to Data You’ll write a simple function that takes a parsed query, looks up the relevant data from your DataFrame, and returns the answer.

In [6]:
def get_financial_metric(company, metric, year, growth_data, ratios_data):
    # Convert the metric to a column name format (reverse the cleanup process)
    column_map = {
        'revenue growth': 'Revenue Growth (%)',
        'net income growth': 'Net Income Growth (%)',
        'liabilities growth': 'Liabilities Growth (%)',
        'assets growth': 'Assets Growth (%)',
        'cash flow growth': 'Cash Flow Growth (%)',
        'net profit margin': 'Net Profit Margin (%)',
        'return on assets (roa)': 'Return on Assets (ROA) (%)',
        'debt to assets ratio': 'Debt to Assets Ratio (%)',
        'equity to assets ratio': 'Equity to Assets Ratio (%)',
        'debt to equity ratio': 'Debt to Equity Ratio',
        'operating cash flow to revenue ratio': 'Operating Cash Flow to Revenue Ratio (%)',
    }
    
    # Look for the correct column in the data
    column_name = column_map.get(metric)
    
    if column_name is None:
        return "Metric not recognized."
    
    try:
        # Convert the Date column to just the year for comparison
        growth_data['Year'] = pd.to_datetime(growth_data['Date']).dt.year
        ratios_data['Year'] = pd.to_datetime(ratios_data['Date']).dt.year
        
        # Check first in growth_data
        if column_name in growth_data.columns:
            print(f"Looking for {company} {metric} in {year} in growth_data...")
            result = growth_data[(growth_data['Company'].str.contains(company, case=False)) & (growth_data['Year'] == int(year))]
            
            if not result.empty:
                value = result[column_name].values[0]
                return f"{company} {metric} in {year}: {value}"
            else:
                print("No matching data found in growth_data.")
        
        # Check in ratios_data if not found in growth_data
        if column_name in ratios_data.columns:
            print(f"Looking for {company} {metric} in {year} in ratios_data...")
            result = ratios_data[(ratios_data['Company'].str.contains(company, case=False)) & (ratios_data['Year'] == int(year))]
            
            if not result.empty:
                value = result[column_name].values[0]
                return f"{company} {metric} in {year}: {value}"
            else:
                print("No matching data found in ratios_data.")
        
    except KeyError as e:
        print(f"KeyError: {e}")
        return "Data not available for this query."
    
    return "No data found for the specified query."


In [7]:
queries = [
    "What was Tesla's net income growth in 2022?",
    "How much did Apple's revenue growth in 2022?",
    "Show me Microsoft's liabilities growth for 2023.",
    "What is Apple's assets growth in 2022?",
    "Can you tell me Tesla’s cash flow growth for 2023?",
    "What was Microsoft's return on assets (roa) in 2022?",
    "Give me the net profit margin for Tesla in 2021.",
    "How did Apple's operating cash flow to revenue ratio change in 2023?",
    "What is the debt to equity ratio for Microsoft in 2022?",
    "Show the liabilities growth for Tesla in 2023."
]

# Process each query and get the response
for query in queries:
    company, metric, year = extract_entities_from_query(query)
    if company and metric and year:
        response = get_financial_metric(company, metric, year, growth_data, ratios_data)
        print(response)
    else:
        print(f"Could not extract enough information from query: {query}")

Extracted company: Tesla, metric: net income growth, year: 2022
Looking for Tesla net income growth in 2022 in growth_data...
Tesla net income growth in 2022: 127.505
Extracted company: Apple, metric: revenue growth, year: 2022
Looking for Apple revenue growth in 2022 in growth_data...
Apple revenue growth in 2022: 7.7938
Extracted company: Microsoft, metric: liabilities growth, year: 2023
Looking for Microsoft liabilities growth in 2023 in growth_data...
Microsoft liabilities growth in 2023: 3.7595
Extracted company: Apple, metric: assets growth, year: 2022
Looking for Apple assets growth in 2022 in growth_data...
Apple assets growth in 2022: 0.4994
Extracted company: Tesla, metric: cash flow growth, year: 2023
Looking for Tesla cash flow growth in 2023 in growth_data...
Tesla cash flow growth in 2023: -9.9701
Extracted company: Microsoft, metric: return on assets (roa), year: 2022
Looking for Microsoft return on assets (roa) in 2022 in ratios_data...
Microsoft return on assets (roa) 

You can then map financial terms matched in the user query to the appropriate column in your data.

In [8]:
# Loop to allow users to input queries
while True:
    # Get user input
    query = input("Please enter a financial query (or type 'exit' to quit): ")
    
    # Exit the loop if the user types 'exit'
    if query.lower() == 'exit':
        break
    
    # Process the query and extract entities
    company, metric, year = extract_entities_from_query(query)
    
    # Check if the extraction was successful
    if company and metric and year:
        # Get the financial metric and print the result
        response = get_financial_metric(company, metric, year, growth_data, ratios_data)
        print(response)
    else:
        print(f"Could not extract enough information from query: {query}")

Please enter a financial query (or type 'exit' to quit):  What is the debt to equity ratio for Microsoft in 2022?


Extracted company: Microsoft, metric: debt to equity ratio, year: 2022
Looking for Microsoft debt to equity ratio in 2022 in ratios_data...
Microsoft debt to equity ratio in 2022: 1.19


Please enter a financial query (or type 'exit' to quit):  exit
